In [ ]:
# Install ktrain
!pip install ktrain
!pip install openpyxl

In [2]:
# Clone the repository for the movie reviews datasets
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...


In [4]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

C:\Users\User\anaconda3\envs\tflowenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data_train = pd.read_excel('C:\\Users\\User\\Documents\\My_Python_Repositories\\NLP_Projects\\IMDB_Movie_Reviews\\Datasets\\train.xlsx')
data_train

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg
...,...,...
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg


In [7]:
data_test = pd.read_excel('C:\\Users\\User\\Documents\\My_Python_Repositories\\NLP_Projects\\IMDB_Movie_Reviews\\Datasets\\test.xlsx')
data_test

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg
...,...,...
24995,This fanciful horror flick has Vincent Price p...,neg
24996,"The Intruder (L'Intrus), a film directed by Fr...",pos
24997,Holy crap. This was the worst film I have seen...,neg
24998,Clocking in at an interminable three hours and...,neg


In [7]:
#preprocessing using ktrain
(x_train, y_train),(x_test, y_test), preprocess = text.texts_from_df(train_df = data_train, 
                  text_column='Reviews',
                  label_columns='Sentiment',
                  val_df = data_test,
                  maxlen=400,
                  preprocess_mode='bert'
                  )

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [8]:
x_train[0].shape

(25000, 400)

In [9]:
# Bert model. Instantiate the model
model = text.text_classifier(name='bert',
                            train_data = (x_train, y_train),
                            preproc=preprocess)

Is Multi-Label? False
maxlen is 400


C:\Users\User\anaconda3\envs\tflowenv\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [10]:
# Get learning rate. Ktrain can get the optimal learning rate. This is like compiling the model
learner = ktrain.get_learner(model = model,
                            train_data = (x_train, y_train),
                            val_data=(x_test, y_test),
                            batch_size = 6)

In [25]:
# Find and plot the optimal learning rate

# NOTE: this might take a day to run. Comment out to avoid running it. The actual result is 2e-5. However, you will need to run this for other projects
#learner.lr_find()
#learner.lr_plot()

In [11]:
# Train the bert model.
# Epoch refers to the number of times the entire training data is passed through the model
# This is a hyperparameter that determines the process of training the machine learning model
# Set the number of epochs to 1 as it will take a long time to run, however, ideally want to find the ideal number of epochs
learner.fit_onecycle(lr = 2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 64683s 16s/step - loss: 0.2532 - accuracy: 0.8960 - val_loss: 0.1023 - val_accuracy: 0.9678


In [14]:
# Predict on the test data
predictor = ktrain.get_predictor(learner.model, preprocess)

In [15]:
# test the model
data = ['This move was horrible. the plot was really boring. Acting was ok. though',
       'The film really sucked. I want my money back',
       'what a beautiful movie. great plot, great acting. will see it again']

In [16]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [17]:
# Save the model
predictor.save('C:\\Users\\User\\Documents\\Python_Scripts\\NLP_Projects\\IMDB_Movie_Reviews\\bert_model')